[SCP642](https://singlecell.broadinstitute.org/single_cell/study/SCP642/cd8-lymphocytes-are-critical-for-early-control-of-tuberculosis-in-macaques#study-download)

[PMID: 37843832](https://pubmed.ncbi.nlm.nih.gov/37843832/)

 **depleted innate and/or adaptive CD8+ lymphocytes in macaques**

 PMCID: PMC10579699 (available on 2024-04-16) 真难受2024年3月6日16:31:34

 treatment: CD8a,CD8b,IgG
 
 看不了文献,不知道时怎么清除CD8的

In [ ]:
import sys
from pathlib import Path
p_root = Path('link','res_publish')

None if str(p_root) in sys.path else sys.path.append(str(p_root))
print("cd '{}'".format(Path('.').absolute()))

In [ ]:
from func import *

In [ ]:
df_meta = pd.read_csv("formatted_metadata.csv", header=[0, 1], index_col=0)
df_meta.columns = [i[0] for i in df_meta.columns]

# print(*df_meta.columns, sep='\n')
# for i in df.columns:
# for i in 'treatment,tb_infection_dose,lung_region'.split(','):
#     print('\n',i.ljust(70,'-'))
#     display(df_meta[i].value_counts())
df_meta = df_meta.loc[:,
                      'species__ontology_label,disease__ontology_label,organ__ontology_label,General_Celltypes,treatment,tb_infection_dose,CFU_TB_per_granuloma,total_thoracic_CFU,lung_region'.split(',')]

df_meta = df_meta.rename(
    columns={
        'species__ontology_label': 'species',
        'disease__ontology_label': 'disease',
        'organ__ontology_label': 'organ'})
display(df_meta.head(), df_meta.shape)


|||
|:-|:-|
|species__ontology_label|Macaca fascicularis|
|disease__ontology_label|tuberculosis|
|library_preparation_protocol__ontology_label|Seq-Well S3|
|organ__ontology_label|lung|
|sex|unknown|


treatment,tb_infection_dose,CFU_TB_per_granuloma,total_thoracic_CFU,lung_region

CFU_TB_per_granuloma 各个肉芽肿的TB 的CFU

total_thoracic_CFU   整个胸廓的TB 的CFU

lung_region

|  ||  |
| :--: | :--: | :--: |
| 右肺下叶 | RLL | Right Lower Lung |
| 左肺下叶 | LLL | Left Lower Lung |
| 肺尖 | ACC | Apex of Chest |
| 右肺上叶 | RUL | Right Upper Lung |
| 左肺上叶 | LUL | Left Upper Lung |
| 左肺中叶 | LML | Left Middle Lung |

In [ ]:
p_temp = p_cache.joinpath('disease','temp_Mtb_macaque_SCP642.h5ad')
adata = None
if p_temp.exists():
    adata = sc.read_h5ad(p_temp)
else:
    adata = sc.read_csv("UMI_counts_unfiltered.csv.gz").T
    adata.write_h5ad(p_temp)
display(adata)

> h5ad 中不是count

```python
adata = sc.read_h5ad("all_cells.h5ad")
adata = sc.read_h5ad("all_cells.h5ad")
display(adata.obs.head(2),adata.obs.shape,adata.var.head(2),adata.var.shape)
adata.obs.to_csv('all_cells_obs.csv',index=True)
temp = adata.X[:100,:100]
temp[temp > 0]
# array([0.63858855, 1.007928  , 0.7136451 , ..., 1.1508739 , 0.70140564,
#        0.34567025], dtype=float32)
```



In [ ]:
df_obs = pd.read_csv('all_cells_obs.csv', index_col=0)
# Gran State --?
# print(*df_obs.columns,sep='\n')
# display(df_obs[''].value_counts())
df_obs = df_obs.loc[:,
                    'batch,treatment,infection dose,lung region,CFU/granuloma,total thoracic CFU,General Celltypes'.split(',')]
df_obs = df_obs.rename(columns={'CFU/granuloma': 'CFU_TB_per_granuloma',
                       'General Celltypes': 'General_Celltypes',
                       'infection dose': 'tb_infection_dose',
                       'lung region': 'lung_region',
                       'total thoracic CFU': 'total_thoracic_CFU'})
display(df_obs.head(2), df_obs.shape,
        pd.Series(adata.obs.index.isin(df_obs.index)).value_counts())

In [ ]:
display(df_obs.head(2), df_meta.head(2))

In [ ]:
display(
    np.intersect1d(df_obs.columns, df_meta.columns),
    np.setdiff1d(df_obs.columns, df_meta.columns),
    np.setdiff1d(df_meta.columns, df_obs.columns)

)

df_obs = df_meta.loc[:, []].join(df_obs)
for i in np.intersect1d(df_obs.columns, df_meta.columns):
    print('df_obs == df_meta all {} key {} '.format((df_obs[i]==df_meta[i]).all(),i))

df_obs = df_obs.join(df_meta.loc[:,np.setdiff1d(df_meta.columns, df_obs.columns)])
display(df_obs.head(2),df_obs.shape)

In [ ]:
display(adata.obs.head(2),adata.obs.shape,df_obs.head(2),df_obs.shape)
print('all of adata.obs.index is in df_obs.index\n{} '.format(df_obs.index.isin(adata.obs.index).all()))

In [ ]:
display(adata.obs.shape)
adata = adata[adata.obs.index.isin(df_obs.index)]
adata.obs = adata.obs.loc[:,[]].join(df_obs)
display(adata.obs.shape,adata.obs.head(2))

In [ ]:
display(adata.obs.head(2))
group_agg(adata.obs,'disease,treatment,tb_infection_dose,batch'.split(','),{'batch':['count']},reindex=False)

In [ ]:
adata.obs['_batch'] = adata.obs.apply(lambda row:'{treatment}_{tb_infection_dose}_{batch}'.format(**row),axis= 1)

adata.obs['cell_type'] = adata.obs['General_Celltypes'].map(
    lambda x:{'Macrophages/Monocytes':'macrophage'}.setdefault(x,''))

In [ ]:
print(*adata.obs.columns,sep='\n')

In [ ]:
adata.obs = adata.obs.loc[
:,'_batch,treatment,tb_infection_dose,batch,cell_type,species,disease,organ,lung_region,General_Celltypes,CFU_TB_per_granuloma,total_thoracic_CFU'.split(',')
]
display(display(adata.obs['General_Celltypes'].value_counts()),adata.obs['cell_type'].value_counts())
adata_marc = adata[adata.obs['cell_type'] == 'macrophage']
adata_marc.obs['macr_cell_type'] = adata_marc.obs['cell_type']
display(adata_marc,adata_marc.obs['cell_type'].value_counts(),
       adata_marc.obs['macr_cell_type'].value_counts())

In [ ]:
h5ad_to_mtx(adata_marc,p_cache.joinpath(
        'disease',
        'Mtb_ma_macrophage_delCD8_SCP642'))


In [ ]:
print("\n[finish]\n".center(100,"-"))